## Spotipy

In [4]:
import spotify_lyrics_scraper as spotify

# Ottieni il token
token = spotify.getToken("AQBSz33_ODmldHiskIdlUiuu4yS92-tR4swS5FeqJuHYfBNUdEs3j0mWK4Hmo9wDc3eTK6IRAE0xFoXnH94CkXHKUWvZuFLTdTxyBdOhLCtSyhZLRWM21tgwJDAI1MxFAA_dEPWMtVMxGd9GefWoL4JbtxCXlsfKvk0CF5zpBQ1m_JskXi7hHLj-wLJBlNdTlpdkeA_LZwBCZG5yQowbiE4hO8K5")

# Ottieni i testi della canzone
lyrics_data = spotify.getLyrics(token, songName="Mr Brightside")

# Controlla se la richiesta è andata a buon fine
if lyrics_data['status']:
    # Estrai le linee di testo
    lyrics_lines = lyrics_data['message']['lyrics']['lines']
    
    # Stampa solo il testo della canzone
    for line in lyrics_lines:
        print(line['words'])
else:
    print("Errore nel recupero dei testi.")


I'm coming out of my cage, and I've been doing just fine
Gotta, gotta be down because I want it all
It started out with a kiss, how did it end up like this?
It was only a kiss, it was only a kiss
Now I'm falling asleep, and she's calling a cab
While he's having a smoke, and she's taking a drag
Now they're going to bed, and my stomach is sick
And it's all in my head, but she's touching his
Chest now, he takes off her dress now
Let me go
♪
I just can't look, it's killing me
And taking control
Jealousy, turning saints into the sea
Swimming through sick lullabies, choking on your alibis
But it's just the price I pay, destiny is calling me
Open up my eager eyes, 'cause I'm Mr. Brightside
♪
I'm coming out of my cage, and I've been doing just fine
Gotta, gotta be down because I want it all
It started out with a kiss, how did it end up like this?
It was only a kiss, it was only a kiss
Now I'm falling asleep, and she's calling a cab
While he's having a smoke, and she's taking a drag
Now they're

In [2]:
# Import necessary libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the pre-trained model and tokenizer
model_name = "j-hartmann/emotion-english-distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Combine all the lyrics lines into a single string
song_lyrics = "\n".join([line['words'] for line in lyrics_lines])

# Tokenize the input text
inputs = tokenizer(song_lyrics, return_tensors="pt", truncation=True, padding=True)

# Get the model outputs
with torch.no_grad():
    outputs = model(**inputs)

# Extract logits and compute probabilities
logits = outputs.logits
probabilities = torch.nn.functional.softmax(logits, dim=1)[0]

# Define the emotions
emotions = [
    'admiration',
    'amusement',
    'anger',
    'love',
    'pain',
    'caring',
    'confusion'
]

# Map emotions to probabilities
emotion_probs = {emotion: float(probabilities[idx]) for idx, emotion in enumerate(emotions)}

# Sort emotions by probability in descending order
sorted_emotions = sorted(emotion_probs.items(), key=lambda x: x[1], reverse=True)

# Print the emotions and their corresponding probabilities
print("Emotion Classification:\n")
for emotion, prob in sorted_emotions:
    print(f"{emotion.capitalize()}: {prob:.4f}")


/Users/ElenaPerego/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Emotion Classification:

Caring: 0.2751
Pain: 0.2427
Confusion: 0.2308
Anger: 0.1135
Love: 0.0595
Admiration: 0.0515
Amusement: 0.0269


## Facial Recognition with DLIB

In [43]:
import dlib
import cv2
import numpy as np
from imutils import face_utils

# Load dlib's face detector and face recognition model
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')  # Download this model file from dlib
face_rec_model = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')  # Download this too

# List to store face encodings and corresponding labels (names)
known_face_encodings = []
known_face_labels = []

# Load and encode your database of faces (use your database here)
def load_known_faces():
    # Example: You have images and labels in a folder (replace with your actual data loading logic)
    # For each face image, detect the face, encode it, and store the encoding with the label (name)
    images = ['/Users/vittoriomocchi/Documents/Vitos/Projects/Emotion-Song-Recommender-System-main/database/Elena.jpg', '/Users/vittoriomocchi/Documents/Vitos/Projects/Emotion-Song-Recommender-System-main/database/Lena.jpg', '/Users/vittoriomocchi/Documents/Vitos/Projects/Emotion-Song-Recommender-System-main/database/Meli.jpg', '/Users/vittoriomocchi/Documents/Vitos/Projects/Emotion-Song-Recommender-System-main/database/Mischa.jpg', '/Users/vittoriomocchi/Documents/Vitos/Projects/Emotion-Song-Recommender-System-main/database/Vito.jpg' ]  # replace with your image paths
    labels = ['Elena', 'Lena', 'Meli', 'Mischa', 'Vito']  # replace with your corresponding names
    
    for img_path, label in zip(images, labels):
        image = cv2.imread(img_path)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = detector(gray_image)

        if len(faces) > 0:
            face = faces[0]  # Use the first detected face
            shape = predictor(gray_image, face)
            face_encoding = np.array(face_rec_model.compute_face_descriptor(image, shape))

            # Add the face encoding and label to the lists
            known_face_encodings.append(face_encoding)
            known_face_labels.append(label)

# Recognize the face from an input image
def recognize_face(input_image_path):
    input_image = cv2.imread(input_image_path)
    gray_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray_image)

    if len(faces) > 0:
        face = faces[0]
        shape = predictor(gray_image, face)
        input_face_encoding = np.array(face_rec_model.compute_face_descriptor(input_image, shape))

        # Compare with known faces
        distances = np.linalg.norm(known_face_encodings - input_face_encoding, axis=1)
        min_distance_idx = np.argmin(distances)

        if distances[min_distance_idx] < 0.6:  # Threshold for recognition
            return known_face_labels[min_distance_idx]
        else:
            return "Unknown"
    return "No face detected"

# Load the known faces (your database)
load_known_faces()

# Now let's recognize a face from a new image
recognized_name = recognize_face('/Users/vittoriomocchi/Documents/Vitos/Projects/Emotion-Song-Recommender-System-main/IMG_5720.jpg')  # replace with the path to the image of you

print(f"Recognized as: {recognized_name}")


Recognized as: Vito


In [45]:
from deepface import DeepFace

# Detect emotions from the input image
def recognize_emotion(input_image_path):
    result = DeepFace.analyze(img_path=input_image_path, actions=['emotion'])
    return result[0]["dominant_emotion"]

# Detect emotions from the image
recognized_emotion = recognize_emotion('/Users/vittoriomocchi/Documents/Vitos/Projects/Emotion-Song-Recommender-System-main/IMG_5720.jpg')

print(f"Detected emotion: {recognized_emotion}")


Detected emotion: sad
